In [ ]:
# do the average/mean https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/2799933550853697/1880776780418274/2202577924924539/latest.html
#https://stackoverflow.com/questions/58507301/word-counter-with-pyspark

In [1]:
import findspark
findspark.init()
import pyspark


from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc=spark.sparkContext

In [2]:
from pyspark.sql import Row
from pyspark.sql.functions import instr,col,column, expr, coalesce, broadcast
from pyspark.sql.functions import count, first, last, min,  sum, avg, sumDistinct,sum_distinct, pow, desc, countDistinct, approx_count_distinct, dense_rank, rank
from pyspark.sql.functions import corr, round, bround, monotonically_increasing_id, mean, format_number, var_pop, stddev_pop,var_samp, stddev_samp, skewness, kurtosis, covar_pop, covar_samp
from pyspark.sql.functions import initcap , lower, upper, lit, ltrim, rtrim, rpad, lpad, trim, regexp_replace, regexp_extract, translate, collect_set, collect_list
from pyspark.sql.functions import when,current_date, current_timestamp, date_add, date_sub, datediff, to_date, months_between, dayofmonth,hour,dayofyear,month,year,weekofyear,date_format
from pyspark.sql.window import Window

from functools import reduce
from operator  import add

### WordCount

In [3]:
book=sc.textFile("data/book.txt")
words=book.flatMap(lambda x : x.split())
wordsAsKey=words.map(lambda x:(x,1))

In [ ]:
words.countByValue()

In [ ]:
wordsAsKey.reduceByKey(lambda x,y: x+y).take(5)               

In [ ]:
wordsAsKey.reduceByKey(add).take(5)                   

In [ ]:
wordsAsKey.groupByKey().map(lambda x: (x[0],len(x[1]))).take(5)               

In [ ]:
wordsAsKey.groupByKey().mapValues(lambda x: len(list(x))).take(5)                

In [ ]:
wordsAsKey.aggregateByKey(0,lambda x,y: x+y,lambda x,y: x+y).take(5)                 

In [ ]:
wordsAsKey.combineByKey(lambda x:x,lambda x,y: x+y,lambda x,y: x+y).take(5)

In [ ]:
wordsAsKey.foldByKey(0,lambda x,y: x+y).take(5)

In [4]:
#This is just extra formating  
wordsAsKey.reduceByKey(lambda x,y: x+y).map(lambda x:(x[1],x[0])).sortByKey(False).map(lambda x:(x[1],x[0])).take(5)
  
wordsAsKey.reduceByKey(lambda x,y: x+y).saveAsTextFile("output/wordcount")

In [ ]:
#word count, print only the files that start with vowel in their individual output file

In [ ]:
book=sc.textFile("data/book.txt")
words=book.flatMap(lambda x : x.split())

def myPartitioner(word):
   if word[0].lower()=='a':return 0
   elif word[0].lower()=='e':return 1
   elif word[0].lower()=='i':return 2
   elif word[0].lower()=='o':return 3
   elif word[0].lower()=='u':return 4
   else:  return 5
    
words.filter(lambda x: x[0].lower()in('a','e','i','o','u')).map(lambda x:(x,1))\
                                .reduceByKey(lambda x,y: x+y).partitionBy(6,myPartitioner).saveAsTextFile("output/wordcountP")